In the data folder there is a data file containing the daily weather summary for new york. 

read it in
count the number of inspections per day
join the result with the daily weather (precipitation (PRCP and SNOW) and temperature (TMAX, TMIN, and TAVG). 

then calculate the average number of daily inspections for days when the precipitation is nonzero, and the average number of daily inspaections for days when the precipitation is zero.

In [1]:
import datetime
import pandas as pd
import os
import numpy as np

In [2]:
os.chdir('../../')
print(os.listdir('.'))

['.DS_Store', 'old_bk_gitconfig', 'solutions', 'code snippets', 'README.md', 'Pipfile', 'notes', '.gitignore', 'exercises', '.git', 'Pipfile.lock', 'using_apis']


In [3]:
os.chdir('exercises/05_joining_data/data') 
print(os.listdir('.'))

['NY_rodent_inspections_sample.csv', 'NY_rodent_inspections_sample_medium.csv', 'ny_weather_data_daily.csv', '1527003.csv.webarchive', 'NY_rodent_inspections_sample_small.csv']


In [4]:
csv_name = 'NY_rodent_inspections_sample.csv'
rodent_dataframe = pd.read_csv(csv_name)
rodent_dataframe.head()

,INSPECTION_TYPE,JOB_TICKET_OR_WORK_ORDER_ID,JOB_ID,JOB_PROGRESS,BBL,BORO_CODE,BLOCK,LOT,HOUSE_NUMBER,STREET_NAME,ZIP_CODE,X_COORD,Y_COORD,LATITUDE,LONGITUDE,BOROUGH,INSPECTION_DATE,RESULT,APPROVED_DATE,LOCATION
0,BAIT,1,PO12965,3,1011470035,1,1147,35,104,WEST 76 STREET,10023,990505,223527,40.780204,-73.977414,Manhattan,10/14/2009 12:00:27 PM,Bait applied,10/14/2009 03:01:46 PM,"(40.7802039792471, -73.9774144709456)"
1,BAIT,2,PO12966,3,1011470034,1,1147,34,102,WEST 76 STREET,10023,990516,223521,40.780188,-73.977375,Manhattan,10/14/2009 12:51:21 PM,Bait applied,10/14/2009 03:02:30 PM,"(40.7801875030438, -73.977374757787)"
2,BAIT,30,PO16966,3,2043370027,2,4337,27,620,THWAITES PLACE,10467,1020110,252216,40.858877,-73.870364,Bronx,11/09/2009 12:59:55 PM,Bait applied,11/10/2009 02:54:52 PM,"(40.8588765781972, -73.8703636422023)"
3,BAIT,31,PO13665,3,2037670077,2,3767,77,1227,WHITEPLAINS ROAD,10472,1022441,242180,40.831321,-73.861994,Bronx,11/09/2009 11:10:16 AM,Bait applied,11/10/2009 02:56:42 PM,"(40.8313209626148, -73.861994089899)"
4,BAIT,38,PO11291,3,1011690057,1,1169,57,2199,BROADWAY,10024,989641,224567,40.783059,-73.980533,Manhattan,11/10/2009 08:40:42 AM,Bait applied,11/17/2009 11:39:11 AM,"(40.7830590725833, -73.9805333640688)"


In [5]:
weather_csv_name = 'ny_weather_data_daily.csv'
weather_dataframe = pd.read_csv(weather_csv_name)
weather_dataframe.head()

,STATION,NAME,LATITUDE,LONGITUDE,ELEVATION,DATE,AWND,FMTM,PGTM,PRCP,...,WT11,WT13,WT14,WT15,WT16,WT17,WT18,WT19,WT21,WT22
0,USW00094789,"JFK INTERNATIONAL AIRPORT, NY US",40.6386,-73.7622,3.4,2009-01-01,8.3,549.0,548.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,USW00094789,"JFK INTERNATIONAL AIRPORT, NY US",40.6386,-73.7622,3.4,2009-01-02,5.7,729.0,734.0,0.0,...,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN
2,USW00094789,"JFK INTERNATIONAL AIRPORT, NY US",40.6386,-73.7622,3.4,2009-01-03,7.4,1252.0,1246.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,USW00094789,"JFK INTERNATIONAL AIRPORT, NY US",40.6386,-73.7622,3.4,2009-01-04,5.2,110.0,109.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,USW00094789,"JFK INTERNATIONAL AIRPORT, NY US",40.6386,-73.7622,3.4,2009-01-05,4.6,1335.0,1334.0,0.3,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN


In [6]:
def format_iso_week(datetime_object):
    the_date = datetime_object.date()
    iso_year, iso_week, iso_weekday = the_date.isocalendar()
    iso_week_string = str(iso_year) + '-W' + str(iso_week)
    return iso_week_string

In [7]:
original_format = '%m/%d/%Y %I:%M:%S %p'

In [8]:
rodent_dataframe['inspection_datetime']   = rodent_dataframe.apply(lambda row: datetime.datetime.strptime(row['INSPECTION_DATE'], original_format), axis=1)

rodent_dataframe['approval_datetime']     = rodent_dataframe.apply(lambda row: datetime.datetime.strptime(row['APPROVED_DATE'],   original_format), axis=1)



rodent_dataframe['inspection_weekday']    = rodent_dataframe.apply(lambda row: row['inspection_datetime'].strftime("%A"), axis=1)

rodent_dataframe['inspection_month']      = rodent_dataframe.apply(lambda row: row['inspection_datetime'].strftime("%B"), axis=1)

rodent_dataframe['isoweek']               = rodent_dataframe.apply(lambda row: format_iso_week(row['inspection_datetime']), axis=1)

rodent_dataframe['days_taken_for_approval'] = rodent_dataframe.apply(lambda row: (row['approval_datetime'].date() - row['inspection_datetime'].date()), axis=1)

rodent_dataframe['just_inspection_date'] = rodent_dataframe.apply(lambda row: (row['inspection_datetime'].date()), axis=1)

In [9]:
rodent_dataframe.head()

,INSPECTION_TYPE,JOB_TICKET_OR_WORK_ORDER_ID,JOB_ID,JOB_PROGRESS,BBL,BORO_CODE,BLOCK,LOT,HOUSE_NUMBER,STREET_NAME,...,RESULT,APPROVED_DATE,LOCATION,inspection_datetime,approval_datetime,inspection_weekday,inspection_month,isoweek,days_taken_for_approval,just_inspection_date
0,BAIT,1,PO12965,3,1011470035,1,1147,35,104,WEST 76 STREET,...,Bait applied,10/14/2009 03:01:46 PM,"(40.7802039792471, -73.9774144709456)",2009-10-14 12:00:27,2009-10-14 15:01:46,Wednesday,October,2009-W42,0 days,2009-10-14
1,BAIT,2,PO12966,3,1011470034,1,1147,34,102,WEST 76 STREET,...,Bait applied,10/14/2009 03:02:30 PM,"(40.7801875030438, -73.977374757787)",2009-10-14 12:51:21,2009-10-14 15:02:30,Wednesday,October,2009-W42,0 days,2009-10-14
2,BAIT,30,PO16966,3,2043370027,2,4337,27,620,THWAITES PLACE,...,Bait applied,11/10/2009 02:54:52 PM,"(40.8588765781972, -73.8703636422023)",2009-11-09 12:59:55,2009-11-10 14:54:52,Monday,November,2009-W46,1 days,2009-11-09
3,BAIT,31,PO13665,3,2037670077,2,3767,77,1227,WHITEPLAINS ROAD,...,Bait applied,11/10/2009 02:56:42 PM,"(40.8313209626148, -73.861994089899)",2009-11-09 11:10:16,2009-11-10 14:56:42,Monday,November,2009-W46,1 days,2009-11-09
4,BAIT,38,PO11291,3,1011690057,1,1169,57,2199,BROADWAY,...,Bait applied,11/17/2009 11:39:11 AM,"(40.7830590725833, -73.9805333640688)",2009-11-10 08:40:42,2009-11-17 11:39:11,Tuesday,November,2009-W46,7 days,2009-11-10


In [10]:
rodent_dataframe['just_inspection_date']

0       2009-10-14
1       2009-10-14
2       2009-11-09
3       2009-11-09
4       2009-11-10
5       2009-11-10
6       2009-11-10
7       2009-11-16
8       2009-11-16
9       2009-11-16
10      2009-11-16
11      2009-11-16
12      2009-11-16
13      2009-11-17
14      2009-11-17
15      2009-11-17
16      2009-11-17
17      2009-11-17
18      2009-11-18
19      2009-11-18
20      2009-11-20
21      2009-11-20
22      2009-11-20
23      2009-11-20
24      2009-11-20
25      2009-11-20
26      2009-11-20
27      2009-11-23
28      2009-11-23
29      2009-11-23
           ...    
9969    2010-07-14
9970    2010-07-14
9971    2010-07-14
9972    2010-07-06
9973    2010-07-14
9974    2010-07-14
9975    2010-07-14
9976    2010-07-14
9977    2010-07-14
9978    2010-07-14
9979    2010-07-14
9980    2010-07-15
9981    2010-07-15
9982    2010-07-15
9983    2010-07-15
9984    2010-07-15
9985    2010-07-15
9986    2010-07-15
9987    2010-07-15
9988    2010-07-15
9989    2010-07-15
9990    2010

In [11]:
# count the number of inspections per day

inspection_day_counts = rodent_dataframe.groupby('just_inspection_date').agg({'just_inspection_date':'count'})
inspection_day_counts.head()

,just_inspection_date
just_inspection_date,
2009-01-16,1
2009-10-14,2
2009-11-09,2
2009-11-10,3
2009-11-16,6


In [12]:
# sorting out and re-indexing the dataframe

inspection_day_counts = inspection_day_counts.rename(columns={"just_inspection_date" : "day_counts"})
inspection_day_counts["date_i"] = inspection_day_counts.index
inspection_day_counts["date_i"] = pd.to_datetime(inspection_day_counts["date_i"])
inspection_day_counts.head()

,day_counts,date_i
just_inspection_date,,
2009-01-16,1,2009-01-16
2009-10-14,2,2009-10-14
2009-11-09,2,2009-11-09
2009-11-10,3,2009-11-10
2009-11-16,6,2009-11-16


In [13]:
weather_only = weather_dataframe[['DATE','PRCP', 'SNOW', 'TMAX', 'TMIN', 'TAVG']]
weather_only.head()

,DATE,PRCP,SNOW,TMAX,TMIN,TAVG
0,2009-01-01,0.0,0.0,-2.2,-8.9,NaN
1,2009-01-02,0.0,0.0,2.8,-5.0,NaN
2,2009-01-03,0.0,0.0,5.0,-1.1,NaN
3,2009-01-04,0.0,0.0,6.7,-3.3,NaN
4,2009-01-05,0.3,0.0,7.2,2.8,NaN


In [14]:
weather_format_descriptor = '%Y-%m-%d'

weather_only["date_w"] = weather_only.apply(lambda row: datetime.datetime.strptime(row['DATE'], weather_format_descriptor), axis=1)
weather_only.head()

/Users/crsharp/.local/share/virtualenvs/solutions-xnM17WUg/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,DATE,PRCP,SNOW,TMAX,TMIN,TAVG,date_w
0,2009-01-01,0.0,0.0,-2.2,-8.9,NaN,2009-01-01
1,2009-01-02,0.0,0.0,2.8,-5.0,NaN,2009-01-02
2,2009-01-03,0.0,0.0,5.0,-1.1,NaN,2009-01-03
3,2009-01-04,0.0,0.0,6.7,-3.3,NaN,2009-01-04
4,2009-01-05,0.3,0.0,7.2,2.8,NaN,2009-01-05


In [15]:
merged = pd.merge(inspection_day_counts,weather_only,how='left', left_on='date_i', right_on='date_w')
merged.head()


,day_counts,date_i,DATE,PRCP,SNOW,TMAX,TMIN,TAVG,date_w
0,1,2009-01-16,2009-01-16,0.0,0.0,-7.2,-11.7,NaN,2009-01-16
1,2,2009-10-14,2009-10-14,0.0,0.0,12.2,5.0,NaN,2009-10-14
2,2,2009-11-09,2009-11-09,0.0,0.0,17.8,5.6,NaN,2009-11-09
3,3,2009-11-10,2009-11-10,0.0,0.0,18.9,10.0,NaN,2009-11-10
4,6,2009-11-16,2009-11-16,0.0,0.0,15.6,9.4,NaN,2009-11-16


In [16]:
merged["total_precipitation"] = merged["SNOW"] + merged["PRCP"]
merged.head()

,day_counts,date_i,DATE,PRCP,SNOW,TMAX,TMIN,TAVG,date_w,total_precipitation
0,1,2009-01-16,2009-01-16,0.0,0.0,-7.2,-11.7,NaN,2009-01-16,0.0
1,2,2009-10-14,2009-10-14,0.0,0.0,12.2,5.0,NaN,2009-10-14,0.0
2,2,2009-11-09,2009-11-09,0.0,0.0,17.8,5.6,NaN,2009-11-09,0.0
3,3,2009-11-10,2009-11-10,0.0,0.0,18.9,10.0,NaN,2009-11-10,0.0
4,6,2009-11-16,2009-11-16,0.0,0.0,15.6,9.4,NaN,2009-11-16,0.0


In [37]:
zero = merged[(merged.total_precipitation == 0.0)]
zero.head()

,day_counts,date_i,DATE,PRCP,SNOW,TMAX,TMIN,TAVG,date_w,total_precipitation
0,1,2009-01-16,2009-01-16,0.0,0.0,-7.2,-11.7,NaN,2009-01-16,0.0
1,2,2009-10-14,2009-10-14,0.0,0.0,12.2,5.0,NaN,2009-10-14,0.0
2,2,2009-11-09,2009-11-09,0.0,0.0,17.8,5.6,NaN,2009-11-09,0.0
3,3,2009-11-10,2009-11-10,0.0,0.0,18.9,10.0,NaN,2009-11-10,0.0
4,6,2009-11-16,2009-11-16,0.0,0.0,15.6,9.4,NaN,2009-11-16,0.0


In [38]:
nonzero = merged[(merged.total_precipitation > 0.0)]
nonzero.head()

,day_counts,date_i,DATE,PRCP,SNOW,TMAX,TMIN,TAVG,date_w,total_precipitation
7,7,2009-11-20,2009-11-20,17.0,0.0,16.1,9.4,NaN,2009-11-20,17.0
10,6,2009-11-25,2009-11-25,0.5,0.0,10.6,9.4,NaN,2009-11-25,0.5
11,7,2009-11-30,2009-11-30,4.3,0.0,12.8,5.0,NaN,2009-11-30,4.3
13,1,2009-12-02,2009-12-02,8.4,0.0,13.9,1.7,NaN,2009-12-02,8.4
14,3,2009-12-03,2009-12-03,15.0,0.0,18.9,10.0,NaN,2009-12-03,15.0


In [39]:
meanzero = zero["day_counts"].mean()
print(meanzero)

54.693548387096776


In [41]:
meannonzero = nonzero["day_counts"].mean()
print(meannonzero)

65.65306122448979
